# Imports

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import cufflinks as cf
import plotly.offline
# cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)
plotly.offline.init_notebook_mode(connected = True)
# pd.options.plotting.backend = "plotly"

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.metrics import f1_score, hinge_loss, roc_curve, auc

from sklearn.preprocessing import label_binarize
from itertools import cycle


In [ ]:
# df = pd.read_csv('/kaggle/input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv')
# tags = ['Футбол','Бокс и ММА','Летние виды','Зимние виды', 'Хоккей', 'Английский футбол']
# sport = df[(df.tags.isin(tags)) | (df.topic == 'Спорт')]
# sport.to_csv('sport_news.csv')
# sport.shape

In [ ]:
df = pd.read_csv(
                #'/kaggle/working/sport_news.csv', 
                '/kaggle/input/for-sport-ru-news/sport_news.csv',
                 index_col=0, parse_dates=['date'])
df.reset_index(drop=True, inplace=True)
df

# Tags - targets

In [ ]:
(df.tags.value_counts()/df.shape[0]).plot.bar()

## URL

Разобьем ссылку на составляющие

In [ ]:
def preprocess_url(x):
    return x.url.split('/')

url = df.apply(preprocess_url, axis=1, result_type='expand')
url.iloc[0]

Нас может заинтересовать здесь последняя составляющая (время уже учтено в наборе данных)

In [ ]:
url = url[[7]]
url

Объединим тэги из ссылок с тематиками статей

In [ ]:
url = url.merge(df.tags, left_index=True, right_index=True)
url.to_csv('urt_to_tag.csv')
url

In [ ]:
# частотность тэгов
url[7].value_counts().sort_values().reset_index(drop=True).plot()

In [ ]:
(url[7].value_counts() > 33).sum()

Большинство тэгов встречаются по небольшому количеству раз. И только чуть больше 200 тэгов являются часто употребляемыми 

Визуализиурем частотные слова по тематикам

In [ ]:
from wordcloud import WordCloud

wcs, tags = [], []
for tag, gr in url.groupby('tags'):
    wcs.append(WordCloud(width = 800, height = 800, 
                background_color ='white', colormap='Blues',
                min_font_size = 10).generate(" ".join(gr[7].values)))
    tags.append(tag)
    
tags[1] = 'Англ. футбол'

In [ ]:
import pickle

pickle.dump(wcs, open('wcs.pickle', 'wb'), )
pickle.dump(tags, open('tags.pickle', 'wb'), )

In [ ]:
fig, axes = plt.subplots(4,2, figsize=(15,30))
font = {'family': 'IBM Plex Sans',
        'color':  'black',
        'weight': 'normal',
        'size': 60,
        }
for ax, wc, tag in zip(axes.ravel(), wcs, tags):
#     plt.figure(figsize = (10, 10), facecolor = 'white', edgecolor='blue') 
    ax.imshow(wc) 
    ax.axis("off") 
    ax.set_title(tag, fontdict=font)
    plt.tight_layout(pad = 0.5) 

Для всех видов спорта частотными словами являются названия видов спорта или имена спорсменов.Но для летних видом спорта частотными словами являются - НХЛ, КХЛ, Овечкин итд (тематика хоккей). Видимо, при загрузке статей на сайт были допущены ошибки и часть статей, тема которых явно соответсвует хоккею, были опубликованы под тематикой летних видов спорта
Проверим, что тэг действительно соответсвует тематике текста

In [ ]:
# тэг khl и тематика - Летние виды
df[(df.url.str.find('khl') != -1) & (df.tags == 'Летние виды')].text.iloc[0]

In [ ]:
# кликабельная ссылка, чтобы удедиться, что информация соотвеует информации на сайте
df[(df.url.str.find('khl') != -1) & (df.tags == 'Летние виды')].iloc[0]

Заченим тематики в ручную -  статьи с основными хоккейными тегами будут с тематикой Хоккей

In [ ]:
to_hockey = ['khl', 'nhl', 'hockey', 
             'ovechkin', 'ovi', 'ove', 
             'malkin', 'tarasenko', 'kovalchuk', 'voynov',
            'fetisov', 'radulov', 'jagr', 'datsyuk', 'bobrovsky',
            'bryzgalov', 'znarok', 'markov', 'nabokov', 'anisimov',
            'yakupov','gagarin', 'gonchar', 'khudobin', 'varlamov',
            'mutko',]

df_ch = df.copy()
df_ch.tags[(url[7].isin(to_hockey))& ((df_ch.tags != 'Все')|(df_ch.tags != 'Все'))] = 'Хоккей'

url_ch = df_ch.apply(preprocess_url, axis=1, result_type='expand')
url_ch.drop([0,1,2,4,5,6, 8], axis=1, inplace=True)
url_ch = url_ch.merge(df_ch.tags, left_index=True, right_index=True)
url_ch.to_csv('urt_to_tag_changed.csv')

In [ ]:
wc = WordCloud(width = 800, height = 800, 
                background_color ='white',
                min_font_size = 10).generate(" ".join(url_ch[url_ch.tags=='Летние виды'][7].values))
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }
plt.figure(figsize=(10,10))
plt.imshow(wc) 
plt.axis("off") 
plt.title('Летние виды', fontdict=font);

In [ ]:
import seaborn as sns

t = pd.concat((df.tags.value_counts()/df.shape[0], df_ch.tags.value_counts()/df.shape[0])).to_frame()
t['type'] = 'changed'
t['type'].iloc[:df.tags.value_counts().shape[0]] = 'ini'

sns.barplot(data=t.reset_index(), x='index', y='tags', hue='type')
plt.xticks(rotation=45);

In [ ]:
df.tags.value_counts().to_csv('tags_count.csv', encoding='windows-1251' )

In [ ]:
df = df[(~df.tags.isna()) & (df.tags != 'Все')]
df_ch = df.copy()
df_ch.tags[(url[7].isin(to_hockey))& (df_ch.tags != 'Все')&(df_ch.tags != 'Футбол')] = 'Хоккей'

# Import for nlp

In [ ]:
!pip install pymystem3

In [ ]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from string import digits

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")
russian_stopwords

In [ ]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = ' '.join(tokens).split()
    tokens = [token for token in tokens\
              if token not in russian_stopwords\
              and token != " " \
              and token.strip(' ') not in digits\
              and token.strip(' ') not in punctuation]
    
    text = " ".join(tokens)
    return text

# Preprocessing title and model

In [ ]:
import IPython
def display(*dfs):
    for df in dfs:
        IPython.display.display(df)

Проверим, что тоектизация и лемматизация выполняется корректно

In [ ]:
display(df.iloc[:10].title.apply(preprocess_text), df.iloc[:10].title )

In [ ]:
title = df.title.apply(preprocess_text)
title.to_csv('prepocess_title.csv')

Создадим мешок слов для признаков. Тематики пронумерум. И создадим hold out  выборку для финальной проверки

In [ ]:
# counter = TfidfVectorizer(ngram_range=(1,1))
counter = CountVectorizer(ngram_range=(1,1))
title_preproc = counter.fit_transform(title)
assert title_preproc.shape[0] == df.shape[0]

le = LabelEncoder()
y = le.fit_transform(df.tags)
int_to_class = {i: cl for i,cl in enumerate(le.classes_)}

x_train ,x_test, y_train, y_test = train_test_split(title_preproc, y,
                                                    random_state=2020, stratify=y)

In [ ]:
x_train.shape

In [ ]:
%%time
model = SGDClassifier(loss='log', random_state=2020, n_jobs=-1, class_weight='balanced',
                     alpha=1*10e-5)
model.fit(x_train, y_train)

In [ ]:
print(classification_report(y_test, model.predict(x_test)))
print(f1_score(y_test, model.predict(x_test), average='macro'))
plot_confusion_matrix(model, x_test, y_test)

In [ ]:
pd.DataFrame(confusion_matrix(y_test, model.predict(x_test))).to_csv('cm_title.csv')

Используя только названия статьей уже можно получить хороший бейзлайн в 0.8 для f1

# Preprocessing text and model

In [ ]:
# %%time
# mystem = Mystem() 
# text = df.text.apply(preprocess_text)
# text.to_csv('text.csv')

In [ ]:
text = pd.read_csv('/kaggle/input/for-sport-ru-news/text.csv', index_col=0)

le = LabelEncoder()
y = le.fit_transform(df.tags)
x_train, x_test, y_train, y_test = train_test_split(text.text, y,
                                                    random_state=2020, stratify=y)

counter = TfidfVectorizer(ngram_range=(1,1), min_df=3)
x_train_tf = counter.fit_transform(x_train.values)
x_test_tf = counter.transform(x_test)
x_train_tf.shape


Попробуем уменьшить пространство признаков с помощью PCA, а заодно визуализируем датасет по тематикам

In [ ]:
from sklearn.decomposition import SparsePCA, TruncatedSVD
import plotly.express as px


pca = TruncatedSVD(n_components=1000)
pca.fit(x_train_tf)

In [ ]:
pd.Series(pca.explained_variance_ratio_).plot()

In [ ]:
pd.Series(pca.explained_variance_ratio_).diff().plot(xlim=(0,50))

Первые 20 компонент имеют наибольшую дисперсию

In [ ]:
pca = TruncatedSVD(n_components=20)

x_pca = pca.fit_transform(x_train_tf)
x_pca.shape

In [ ]:
to_pca = pd.DataFrame(x_pca).merge(pd.Series(y_train,name='class'), left_index=True, right_index=True)
to_pca['class'] = to_pca['class'].map(int_to_class)
to_pca.to_csv('text_pca20.csv')

In [ ]:
px.scatter_3d(to_pca, x=1, y=0, z=2, color='class', 
              title=f'Total Explained Variance: {pca.explained_variance_ratio_.sum() * 100:.2f}%',
              labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},  
              category_orders={'class': np.sort(to_pca['class'].unique()).tolist()},
              color_discrete_sequence= px.colors.sequential.Plasma_r
              )

Тематика Бокса и ММА имеет явное отличие в пространсве признако и поэтому её будет проще всего предсказать. Остальные тематики смешаны друг относительно друга и находятся в одной плоскости 

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%%time

x_train, x_test, y_train, y_test = train_test_split(text.text, y,
                                                    random_state=2020, stratify=y)
pipeline = Pipeline([('counter', TfidfVectorizer(ngram_range=(1,2), min_df=10)),
#                     ('pca', TruncatedSVD(n_components=20)),
                    ('model', SGDClassifier(loss='log', random_state=2020, n_jobs=-1, class_weight='balanced',
                     alpha=0.5*10e-6))
#                      ('model', SGDClassifier( random_state=2020, n_jobs=-1, class_weight='balanced',
#                      alpha=0.5*10e-5))
#                      ('model', RandomForestClassifier(n_estimators=150, class_weight='balanced', n_jobs=1,
#                                                       max_depth=10,
#                                                       min_samples_split=3, #max_features='log2'
#                                                      ))                     
                    ])

results = cross_validate(pipeline, x_train, y_train,
                    scoring='f1_macro', cv=3,
                    return_train_score=True, n_jobs=-1)
results, results['test_score'].mean(), results['train_score'].mean()

Лучший результат модели на кросс валидации - 0.891. Проверим итоговое значение метрики на отложенной выборке

In [ ]:
pipeline.fit(x_train, y_train)
pred = pipeline.predict(x_test)
print(classification_report(y_test, pred))
print(f1_score(y_test, pred, average='macro'))
plot_confusion_matrix(pipeline, x_test, y_test);

In [ ]:
int_to_class

In [ ]:
cm = pd.DataFrame(confusion_matrix(y_test, pipeline.predict(x_test)))
cm.columns = cm.columns.map(int_to_class)
cm.index = cm.index.map(int_to_class)
cm.to_csv('cm_text.csv')

# Missclassification study
Изучим по подробней статьи на которых классификатор ошибается

In [ ]:
pred = pipeline.predict(x_test)
miss_class_x = x_test[y_test != pred]
miss_class_y = y_test[y_test != pred]
miss_class_pred = pred[y_test != pipeline.predict(x_test)]
idx = y_test != pipeline.predict(x_test)

prediction = pd.concat((df.loc[miss_class_x.index][['title', 'text']].reset_index(drop=True), pd.Series(miss_class_y).map(int_to_class), pd.Series(miss_class_pred).map(int_to_class)), axis=1)
prediction.rename(columns={0: 'true', 1:'predict'}, inplace=True)
prediction.to_csv('prediction.csv')

In [ ]:
fig = plt.figure(figsize=(10,5))
fig.suptitle('The probability distribution of main class ')
ax = plt.subplot(121)
pd.Series(pipeline.predict_proba(x_test.loc[idx]).max(axis=1)).plot(kind='hist', bins=20, ax=ax, 
                                                                    title='in miss-classification sample')

ax = plt.subplot(122)
pd.Series(pipeline.predict_proba(x_test.loc[~idx]).max(axis=1)).plot(kind='hist', bins=20, ax=ax,
                                                                    title='in right classification samples')

Для правильно классифицированных статей характерны высокие значения вероятности. 

In [ ]:
pred_miss = pd.Series(pipeline.predict_proba(x_test.loc[idx]).max(axis=1))
pred_right = pd.Series(pipeline.predict_proba(x_test.loc[~idx]).max(axis=1))

In [ ]:
threshold = 0.90
print(f'classificate as right in missclassification:\t {(pred_miss > threshold).sum() / pred_miss.shape[0] * 100}')
print(f'not sure for right classification:\t\t {(pred_right < threshold).sum() / pred_right.shape[0] * 100}')

In [ ]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [ ]:
int_to_class

In [ ]:
i = pd.Series(pred_miss)[(0.80 < pred_miss) ].index
t = pd.concat((x_test.iloc[idx].iloc[i].reset_index(), 
               pd.DataFrame(pipeline.predict_proba(x_test.iloc[idx].iloc[i])),
               pd.Series(y_test[idx][i], name='true')), 
         axis=1)

t['pred'] = t.iloc[:, -8:-1].idxmax(1).map(int_to_class)
t['true'] = t['true'].map(int_to_class)
t['prob'] = t.iloc[:, -10:-3].max(1)
t[['text', 'pred', 'true', 'prob']].sort_values('prob', ascending=False).style.format({'text': '{:.200s}'})
# t.style.apply(highlight_max, subset=[0,1,2,3,4,5,6], axis=1)

In [ ]:
p = pipeline.predict_proba(x_test)
p[p.max(axis=1) < 0.90].shape[0],p[p.max(axis=1) < 0.90].shape[0]/p.shape[0], p.shape[0] - p[p.max(axis=1) < 0.90].shape[0]

In [ ]:
t = pd.concat((x_test.reset_index(), pd.DataFrame(p), pd.Series(y_test, name='true')), 
              axis=1)
t['max_prob'] = t.iloc[:, -8:-1].max(1)
t = t[t.max_prob < 0.90]
t['pred'] = t.iloc[:, -9:-2].idxmax(1).map(int_to_class)
t['true'] = t['true'].map(int_to_class)
t.max_prob.plot(kind='hist', bins=20)
# t[['text', 'pred', 'true']].style.format({'text': '{:.200s}'})
# t.style.apply(highlight_max, subset=[0,1,2,3,4,5,6], axis=1)

# With changed data

In [ ]:
df_ch = df.copy()
df_ch.tags[(url[7].isin(to_hockey))& ((df_ch.tags != 'Все')&(df_ch.tags != 'Футбол'))] = 'Хоккей'

In [ ]:
df_ch.tags.value_counts() - df.tags.value_counts()

In [ ]:
le = LabelEncoder()
y = le.fit_transform(df_ch.tags)
x_train, x_test, y_train, y_test = train_test_split(text.text, y,
                                                    random_state=2020, stratify=y)

In [ ]:
%%time
pipeline = Pipeline([('counter', TfidfVectorizer(ngram_range=(1,2), min_df=10)),
#                     ('pca', TruncatedSVD(n_components=20)),
                    ('model', SGDClassifier(loss='log', random_state=2020, n_jobs=-1, class_weight='balanced',
                     alpha=0.1*10e-6))
#                      ('model', RandomForestClassifier(n_estimators=150, class_weight='balanced', n_jobs=1,
#                                                       max_depth=10,
#                                                       min_samples_split=3, #max_features='log2'
#                                                      ))                     
                    ])

results = cross_validate(pipeline, x_train, y_train,
                    scoring='f1_macro', cv=3,
                    return_train_score=True, n_jobs=-1)
results, results['test_score'].mean(), results['train_score'].mean()

In [ ]:
pipeline.fit(x_train, y_train)
pred = pipeline.predict(x_test)
print(classification_report(y_test, pred))
print(f1_score(y_test, pred, average='macro'))
plot_confusion_matrix(pipeline, x_test, y_test);

Сущетсвеннхы изменений в классификации нет и чистка лейблов не дает преимущества

In [ ]:
int_to_class